---
# **LAB 10 - Parallel patterns**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

## Bash and data setup

In [ ]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

In [ ]:
!source /root/.bashrc

Clone GPUcomputing site on github...

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

Define some paths...

In [ ]:
# path setup
!mkdir -p /content/GPUcomputing/lab10
%cd /content/GPUcomputing/lab10
!mkdir -p BFS
!mkdir -p coloring
!mkdir -p sorting

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

# ▶️ DeviceQuery

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)
!nvcc /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery

Check whether the device can transfer in both directions simultaneously

In [ ]:
%%cu
#include <stdio.h>

int main(void) {

  cudaDeviceProp dProp;
	cudaGetDeviceProperties(&dProp, 0);

  // Shows whether the device can transfer in both directions simultaneously
  printf("Device %s capable of simultaneous CPU-to-GPU and GPU-to-CPU datatransfers\n", dProp.deviceOverlap ? "IS": "NOT");
  return 0;
}

# ✅ BFS

In [ ]:
%%writefile BFS/testBFS.cu

#include <stdio.h>
#include <stdlib.h>
#include "../../utils/graph/graph.h"
#include "../../utils/common.h"

__global__ void print_d(GraphStruct*, bool);

__global__ void initBFS(GraphStruct *G, bool *Fa, bool *Xa, unsigned int n) {
	int nodeID = threadIdx.x + blockIdx.x * blockDim.x;

	if (nodeID > n)
		return;
  
  // set Fa and Xa vectors to false
  Fa[nodeID] = false;
  Xa[nodeID] = false;
	
}

/**
 * Kernel: The BFS frontier corresponds to all the nodes
 *         being processed at the current level
 */
__global__ void cudaBFS(GraphStruct *G, bool *Fa, bool *Xa, int *Ca, bool *done, int n) {

	int nodeID = threadIdx.x + blockIdx.x * blockDim.x;   // node ID

	if (nodeID > n)
		return;

	if (Fa[nodeID]) {
		*done = false;
		Fa[nodeID] = false;
		Xa[nodeID] = true;
		int deg = G->cumDegs[nodeID + 1] - G->cumDegs[nodeID];
		int start = G->cumDegs[nodeID];
		for (int i = 0; i < deg; i++) {
			int neighID = G->neighs[start + i];
			if ( !Xa[neighID] ) {
				Ca[neighID] = Ca[nodeID] + 1;
				Fa[neighID] = true;
			}
		}
	}
}

/**
 * MAIN: BFS test both CPU & GPU
 */
int main() {

	int BLOCK_SIZE = 512;
	unsigned int N = 20;                // number of nodes for random graphs
	float prob = .5;                     // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed
	bool GPUEnabled = 1;
	Graph graph(N,GPUEnabled);

	// generate a random graph
	graph.randGraph(prob,eng);
	
	printf("** Graph done! \n");

	// get the graph struct
	GraphStruct *G = graph.getStruct();
	print_d<<<1,1>>>(G, 1);

	// setup vars for BFS
	bool *Fa, *Va, *Xa;
	int *Ca;
	CHECK(cudaMallocManaged((void **)&Fa, N* sizeof(bool)));
	CHECK(cudaMallocManaged((void **)&Va, N* sizeof(bool)));
	CHECK(cudaMallocManaged((void **)&Xa, N* sizeof(bool)));
	CHECK(cudaMallocManaged((void **)&Ca, N* sizeof(int)));

	// set the source
	int source = 0;
	Fa[source] = true;

	bool done;
	bool *d_done;
	cudaMalloc((void **)&d_done, sizeof(bool));
	int count = 0;
	printf("** BFS array size N = %d\n",N);

	// events to measure time
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	int nThreads = N;
	dim3 block(min(nThreads, BLOCK_SIZE));
	dim3 grid((nThreads + block.x - 1) / block.x);
	cudaEventRecord(start);
	initBFS<<<grid, block>>>(G, Fa, Xa, N);
	CHECK(cudaDeviceSynchronize());
	Fa[0] = true;
	Ca[0] = 0;
	do {
		count++;
		done = true;
		cudaMemcpy(d_done, &done, sizeof(bool), cudaMemcpyHostToDevice);
		cudaBFS<<<grid, block>>>(G, Fa, Xa, Ca, d_done, N);
		cudaMemcpy(&done, d_done, sizeof(bool), cudaMemcpyDeviceToHost);
	} while (!done);
	CHECK(cudaDeviceSynchronize());
	CHECK(cudaEventRecord(stop));
	CHECK(cudaEventSynchronize(stop));
	float milliseconds;
	cudaEventElapsedTime(&milliseconds, start, stop);
	float GPUtime = milliseconds / 1000.0;
	printf("   elapsed time:   %.5f (sec)\n", GPUtime);
	printf("   Number of times the kernel is called : %d \n", count);

	int max_Ca = 0;
	printf("\nCost: ");
	for (int i = 0; i < N; i++) {
		if (Ca[i] > max_Ca)
			max_Ca = Ca[i];
	}
	printf("max Ca = %d\n", max_Ca);

	return 0;
}

In [ ]:
# Compilazione ed esecuzione
!rm -rf *.o
!nvcc -arch=sm_37 -dc BFS/testBFS.cu ../utils/graph/graph.cpp ../utils/graph/graph_d.cu
!nvcc -arch=sm_37 testBFS.o graph.o graph_d.o -o testBFS
!./testBFS

# ✅ Luby coloring


In [ ]:
%%writefile coloring/coloring.h
#pragma once

#include <curand_kernel.h>
#include "../../utils/graph/graph.h"
#include "../../utils/common.h"

/**
 *  graph coloring struct (colors are: 1,2,3,..,k)
 */

struct Coloring {
	bool		uncoloredNodes;
	uint		numOfColors;
	uint	*	coloring;   // each element denotes a color
	//~Coloring() {delete[] coloring;}
};

Coloring* LubyGreedy(GraphStruct*);
void printColoring (Coloring*, GraphStruct*, bool);
__global__ void LubyJPcolorer (Coloring*, GraphStruct*, uint*) ;
__global__ void init(uint seed, curandState_t*, uint*, uint);
__global__ void findIS (Coloring*, GraphStruct*, uint*);
__global__ void print_d(GraphStruct*, bool);

In [ ]:
%%writefile coloring/Luby.cu

#include <iostream>
#include "coloring.h"
#include "../../utils/graph/graph_d.h"
#include "../../utils/common.h"

using namespace std;

#define THREADxBLOCK 128

Coloring* LubyGreedy(GraphStruct *str) {
	// set coloring struct

	Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);

	// start coloring (dyn. parall.)
	LubyJPcolorer <<< 1, 1 >>> (col, str, weigths);

	cudaFree(states);
	cudaFree(weigths);
	return col;
}

/**
 * find an IS
 */
__global__ void findIS (Coloring* col, GraphStruct *str, uint* weights) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		col->coloring[idx] = col->numOfColors;
	}
	else
		col->uncoloredNodes = true;
}


/**
 *  this GPU kernel takes an array of states, and an array of ints, and puts a random int into each
 */
__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx > n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}


/**
 * Luby IS & Jones−Plassmann colorer
 */
__global__ void LubyJPcolorer (Coloring* col, GraphStruct *str, uint* weights) {
	dim3 threads (THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

	// loop on ISs covering the graph
	col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weights);
		cudaDeviceSynchronize();
	}
}

/**
 * Print the graph (verbose = 1 for "verbose print")
 * @param verbose print the complete graph
 */
void printColoring (Coloring* col, GraphStruct* str, bool verbose) {
	node n = str->nodeSize;
	cout << "** Graph (num node: " << n << ", num edges: " << str->edgeSize << ")" << endl;
	cout << "** Coloring (num colors: " << col->numOfColors << ")" << endl;
	if (verbose) {
		for (int i = 1; i <= col->numOfColors; i++) {
			cout << "   color(" << i << ")" << "-> ";
			for (int j = 0; j < n; j++)
				if (col->coloring[j] == i)
					cout << j << " ";
			cout << "\n";
		}
		cout << "\n";
	}
}




### TEST on Luby


Graph print layout:
```
** Graph (num node: 10, num edges: 46)
      (min deg: 3, max deg: 7, mean deg: 4.6, connected: 1)
      node(0)[5]-> 1 2 3 7 9
      node(1)[3]-> 0 2 9
      node(2)[5]-> 0 1 3 5 8
      node(3)[7]-> 0 2 4 5 6 7 9
      node(4)[5]-> 3 6 7 8 9
      node(5)[5]-> 2 3 6 8 9
      node(6)[4]-> 3 4 5 8
      node(7)[3]-> 0 3 4
      node(8)[4]-> 2 4 5 6
      node(9)[5]-> 0 1 3 4 5
```


Coloring print layout:
```
** Graph (num node: 10, num edges: 36)
** Coloring (num colors: 6)
    color(1)-> 1 3 8
    color(2)-> 0 6
    color(3)-> 4 7
    color(4)-> 9
    color(5)-> 5
    color(6)-> 2
```




In [ ]:
%%writefile coloring/test_Luby.cu

#include "coloring.h"

int main(void) {
	unsigned int n = 10000;		 // number of nodes for random graphs
	float prob = .01;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

	// print small graph
	if (n <= 20) {
		graph.print(true);  // CPU print
		print_d<<< 1, 1 >>>(str, true);  // GPU print
	}
	
	// GPU Luby-JP greedy coloring
	Coloring* col = LubyGreedy(str);
	cudaDeviceSynchronize();
	printColoring(col, str, 1);

	return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 -dc coloring/test_Luby.cu coloring/Luby.cu ../utils/graph/graph.cpp ../utils/graph/graph_d.cu
!nvcc -arch=sm_37 test_Luby.o Luby.o graph.o graph_d.o -o testLuby 
!./testLuby

# ✅ MergeSort

In [ ]:
%%writefile sorting/mergesort.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include "../../utils/common.h"

#define ARRAY_SIZE 64

void check_up_sorting(int *, unsigned);
void random_array(int *, unsigned, int);
void printArray(int *, int, int);
void mergeSort(int *, int, int);
void merge(int *, int, int, int);
void arrayCopy(int *, const int *, const int);


/**
 * Kernel: mergeSort with seq. merge
 */
__global__ void cudaMergeSort(int *array, int *sorted, int n, int chunk) {

	int start = chunk * (threadIdx.x + blockIdx.x * blockDim.x);
	if (start > n - chunk)
		return;

	int mid = start + chunk / 2;
	int end = start + chunk;
	int i = start, j = mid, k = start;

	//cudaMerge(array, sorted, start, mid, end);
	while (i < mid && j < end) 
		if (array[i] <= array[j]) 
			sorted[k++] = array[i++];
		else 
			sorted[k++] = array[j++];

	// Copy the remaining elements array[i] if there are any
	while (i < mid)
		sorted[k++] = array[i++];

	// Copy the remaining elements of array[j] if there are any
	while (j < end)
		sorted[k++] = array[j++];
}

/**
 * A iterative binary search function. It returns the location p of
 * the first element in r-length arr[0..r-1] greater than x
 */
__device__ int binarySearch(int arr[], int x, int k, bool UP) {
	int l = 0, r = k;

	while (l < r) {
		int m = (l+r)/2;
		if (UP) {     // for upper chunk B
			if (arr[m] <= x) l = m + 1;
			else r = m;
		}
		else {   // for lower chunk A
			if (arr[m] < x) l = m + 1;
			else r = m;
		}
	}

	return l;
}

/**
 * Kernel: mergeSort with many threads. Each thread deals with 2 elements:
 *  A[i] in first chunk and the corresponding B[i] in the second chunk
 */
__global__ void cudaMergeSortMulti(int *array, int *sorted, int n, int k) {
	// k = 1,2,4,8,16,..., 2^m chunk dims
	int tid = threadIdx.x + blockIdx.x * blockDim.x;
	int j = tid % k;
	int l = (tid - j)*2;  // first element of the fisrt chunk
	int i = l + j;        // A[i] first chunk   [][][*][] and  B[i+k] [][][*][]

	if (k == 1) {
		l = 2*tid;
		i = l;
	}

	// find the relative position of x within B[*]
	int x = array[i];
	int p = binarySearch(array+l+k, x, k, 1);
	sorted[i+p] = x;

	// find the relative position of y within A[*]
	int y = array[i+k];
	p = binarySearch(array+l, y, k, 0);
	sorted[i+p] = y;
}

/*
 * MAIN
 */
int main(int argc, char** argv) {

	// Create the vector with the specified size and situation
	int *orig, *array, *sorted;
	int N = 4*1024*1024;         // must be a power of 2
	int BLOCK_SIZE = 32;


	printf("*** Sorting array size N = %d\n",N);

	// events to measure time
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	// managed memory
	CHECK(cudaMallocManaged((void **)&array, N * sizeof(int)));
	CHECK(cudaMallocManaged((void **)&sorted, N * sizeof(int)));

	// random instance
	orig = (int *) malloc(N * sizeof(int));
	random_array(orig, N, 1);
	arrayCopy(array, orig, N);
	// printArray(array,N,16);

	/*****************************************************
	 *                      CPU                          *
	 *****************************************************/
	
  printf("*** CPU processing...\n");
	double startTm = seconds();
	mergeSort(array, 0, N);
	double CPUtime = seconds() - startTm;
	printf("   CPU elapsed time: %.5f (sec)\n", CPUtime);
	check_up_sorting(array, N);

	/*****************************************************
	 *              ONE THREAD x chunk                   *
	 *****************************************************/
	
  printf("\n*** GPU ONE THREAD x chunk processing...\n");
	arrayCopy(sorted, orig, N); // start from step 2
	bool array2sorted = false;
	CHECK(cudaEventRecord(start));
	for (int chunk = 2; chunk <= N; chunk *= 2) {
		int nThreads = N / chunk;
		dim3 block(min(nThreads, BLOCK_SIZE));
		dim3 grid((nThreads + block.x - 1) / block.x);

		if (array2sorted)
			cudaMergeSort<<<grid, block>>>(array, sorted, N, chunk);
		else
			cudaMergeSort<<<grid, block>>>(sorted, array, N, chunk);
		array2sorted = !array2sorted;
	}
	CHECK(cudaDeviceSynchronize());
	CHECK(cudaEventRecord(stop));
	CHECK(cudaEventSynchronize(stop));
	float milliseconds;
	cudaEventElapsedTime(&milliseconds, start, stop);
	float GPUtime = milliseconds / 1000.0;
	printf("   elapsed time:   %.5f (sec)\n", GPUtime);
	printf("   speedup vs CPU: %.2f\n", CPUtime / GPUtime);

	check_up_sorting(sorted, N);

	/*****************************************************
	 *              MULTI THREAD x chunk                 *
	 *****************************************************/
	printf("\n*** GPU MULTI THREAD x chunk processing...\n");
	arrayCopy(array, orig, N);
	array2sorted = false;

	// grid set up
	int nThreads = N/2;
	dim3 block(min(nThreads, BLOCK_SIZE));
	dim3 grid((nThreads + block.x - 1) / block.x);
	CHECK(cudaEventRecord(start));
	for (int chunk = 1; chunk <= N/2; chunk *= 2) {
		array2sorted = !array2sorted;
		if (array2sorted)
			cudaMergeSortMulti<<<grid, block>>>(array, sorted, N, chunk);
		else
			cudaMergeSortMulti<<<grid, block>>>(sorted, array, N, chunk);
	}
	CHECK(cudaDeviceSynchronize());
	CHECK(cudaEventRecord(stop));
	CHECK(cudaEventSynchronize(stop));
	cudaEventElapsedTime(&milliseconds, start, stop);
	float GPUtime1 = milliseconds / 1000.0;
	printf("   elapsed time:        %.5f (sec)\n", GPUtime1);
	printf("   speedup vs CPU:      %.2f\n", CPUtime / GPUtime1);
	printf("   speedup vs GPU mono: %.2f\n", GPUtime / GPUtime1);
	if (!array2sorted) {
		int *swap = sorted;
		sorted = array;
		array = swap;
	}
	check_up_sorting(sorted, N);

//	printArray(array,N,32);
//	printArray(sorted,N,64);

	return 0;

}

/**
 *  Merge the two half into a sorted data
 */
void merge(int arr[], int l, int m, int r) {
	//	printf("merge: l = %d, m = %d, r = %d\n",l,m,r);
	int i, j, k;
	int n1 = m - l + 1;
	int n2 = r - m;

	// Create temp arrays
	//	int L[n1], R[n2];
	int *L = new int[n1];
	int *R = new int[n2];

	// Copy data to temp arrays L[] and R[]
	for (i = 0; i < n1; i++)
		L[i] = arr[l + i];
	for (j = 0; j < n2; j++)
		R[j] = arr[m + 1 + j];

	// Merge the temp arrays back into arr[l..r]

	i = 0; // Initial index of first subarray
	j = 0; // Initial index of second subarray
	k = l; // Initial index of merged subarray

	while (i < n1 && j < n2) {
		if (L[i] <= R[j]) {
			arr[k] = L[i];
			i++;
		} else {
			arr[k] = R[j];
			j++;
		}
		k++;
	}

	// Copy the remaining elements of L[], if there are any
	while (i < n1) {
		arr[k] = L[i];
		i++;
		k++;
	}

	// Copy the remaining elements of R[], if there are any
	while (j < n2) {
		arr[k] = R[j];
		j++;
		k++;
	}
}

// l is for left index and r is right index of the
// sub-array of arr to be sorted
void mergeSort(int arr[], int l, int r) {
	//	printf("mergeSort: l = %d, r = %d\n",l,r);
	if (l < r) {
		int m = (l + r) / 2;

		// Sort first and second halves
		mergeSort(arr, l, m);
		mergeSort(arr, m + 1, r);

		// merge in backtracking step
		merge(arr, l, m, r);
	}
}

/**
 * Function that fills an array with random integers
 * @param int* array Reference to the array that will be filled
 * @param int  size  Number of elements
 */
void random_array(int *array, unsigned size, int seed) {
	srand(seed);
	int i;
	for (i = 0; i < size; i++) {
		array[i] = rand() % size;
	}
}

/**
 * Function that checks whether the sorting is correct
 * @param int* array Reference to the array
 * @param int  size  Number of elements
 */
void check_up_sorting(int array[], unsigned size) {
	bool flag = true;
	for (int i = 0; i < size - 1; i++)
		if (array[i] > array[i + 1]) {
			printf("Sorting error! array[%d]=%d array[%d]=%d\n", i, array[i], i + 1,
					array[i + 1]);
			flag = false;
			break;
		}
	if (flag)
		printf("   Sorting OK!\n");
}

/*
 * Function to print an array
 */
void printArray(int arr[], int size, int k) {
	for (int i = 0; i < size; i++) {
		if (i>0 && k > 0 && i%k==0)
			printf("\n");
		printf("%d ", arr[i]);
	}
	printf("\n\n");
}

/*
 * Function to print an array
 */
void arrayCopy(int dst[], const int src[], const int size) {
	for (int i = 0; i < size; i++)
		dst[i] = src[i];
}



In [ ]:
# for debugging (small vectors)
#!nvcc -arch=sm_37 -DDEBUG sorting/bitonic.cu -o bitonic

# for test on big vectors
!nvcc -arch=sm_37 sorting/mergesort.cu -o mergesort
!./mergesort

# ✅ BitonicSort


 * Gli indici $k$ e $j$ in input hanno significato:
   * $k = 2,4,8,...,2^s=N$
   * $j = 2^{(k-1)}, 2^{(k-2)},...,1$ (parte dalla metà di k e continua a dimezzare)
 * Gli operatori sui bit ^ (XOR) e & (AND) vengono usati per filtrare i thread:
   * $ixj = i^j$  aggiunge o toglie a $i$ una potenza di 2, cioé $ixj = i \pm j$ con $j = 2^a$
   * $i \& k == 0$ vero sse $i \le k$ (sort ascendente) altrimenti sort discendente
 * L'operazione $ixj > i$ significa aggiorna solo quando l'indice $ixj$ fa un salto in avanti di $j=2^a$
 

In [ ]:
%%writefile sorting/bitonic.cu

#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#include "../../utils/common.h"

#define THREADS 512
#define BLOCKS 8*1024

__global__ void bitonic_sort_step(int *a, int j, int k) {
	unsigned int i, ixj;      // Sorting partners i and ixj
	i = threadIdx.x + blockDim.x * blockIdx.x;
	ixj = i ^ j;              // XOR: aggiunge o toglie a i una potenza di 2, j = 2^a

  #ifdef DEBUG
	if (i == 0)
		printf("ROUND: k = %d, j = %d\n", k, j);
  #endif

	if ((ixj) > i) {    // entra solo quando fa un salto di j = 2^a

    // Sort ascending
		if ((i & k) == 0) {
      #ifdef DEBUG
			printf("  UP  (ixj = %d\t    i = %d\t k = %d)   a[ixj] = %d - a[i] = %d\n", ixj, i, k, a[ixj],a[i]);
      #endif
			if (a[i] > a[ixj]) {
				int temp = a[i];
				a[i] = a[ixj];
				a[ixj] = temp;
			}
		}

		// Sort descending
		if ((i & k) != 0) {
      #ifdef DEBUG
			printf("  DOWN  (ixj = %d\t    i = %d\t k = %d)   a[ixj] = %d - a[i] = %d\n", ixj, i, k, a[ixj],a[i]);
      #endif
			if (a[i] < a[ixj]) {
				int temp = a[i];
				a[i] = a[ixj];
				a[ixj] = temp;
			}
		}
	}
}

/*The parameter dir indicates the sorting direction, ASCENDING
 or DESCENDING; if (a[i] > a[j]) agrees with the direction,
 then a[i] and a[j] are interchanged.*/
void compAndSwap(int a[], int i, int j, int dir) {
	if (dir == (a[i] > a[j])) {
		int tmp = a[i];
		a[i] = a[j];
		a[j] = tmp;
	}
}

/*It recursively sorts a bitonic sequence in ascending order,
 if dir = 1, and in descending order otherwise (means dir=0).
 The sequence to be sorted starts at index position low,
 the parameter cnt is the number of elements to be sorted.*/
void bitonicMerge(int a[], int low, int cnt, int dir) {
	if (cnt > 1) {
		int k = cnt / 2;
		for (int i = low; i < low + k; i++)
			compAndSwap(a, i, i + k, dir);
		bitonicMerge(a, low, k, dir);
		bitonicMerge(a, low + k, k, dir);
	}
}

/* This function first produces a bitonic sequence by recursively
 sorting its two halves in opposite sorting orders, and then
 calls bitonicMerge to make them in the same order */
void bitonicSort(int a[], int low, int cnt, int dir) {
	if (cnt > 1) {
		int k = cnt / 2;

		// sort in ascending order since dir here is 1
		bitonicSort(a, low, k, 1);

		// sort in descending order since dir here is 0
		bitonicSort(a, low + k, k, 0);

		// Will merge wole sequence in ascending order
		// since dir=1.
		bitonicMerge(a, low, cnt, dir);
	}
}

/*
 * MAIN: test bitonic sort on CPU and GPU
 */
int main(void) {
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	int N = THREADS*BLOCKS;
	// check
	if (!(N && !(N & (N - 1)))) {
		printf("ERROR: N must be power of 2 (N = %d)\n", N);
		exit(1);
	}
	size_t nBytes = N * sizeof(int);
	int *a = (int*) malloc(nBytes);
	int *b = (int*) malloc(nBytes);

	// fill data
	for (int i = 0; i < N; ++i) {
		a[i] =  rand() % 100; 
		b[i] = a[i];
	}

	// bitonic CPU
	double cpu_time = seconds();
	bitonicSort(b, 0, N, 1);   // 1 means sort in ascending order
	printf("CPU elapsed time: %.5f (sec)\n", seconds()-cpu_time);

	// device mem copy
	int *d_a;
	CHECK(cudaMalloc((void**) &d_a, nBytes));
	CHECK(cudaMemcpy(d_a, a, nBytes, cudaMemcpyHostToDevice));

	// num of threads
	dim3 blocks(BLOCKS, 1);   // Number of blocks
	dim3 threads(THREADS, 1); // Number of threads

	// start computation
	cudaEventRecord(start);
	int j, k;
	// external loop on comparators of size k
	for (k = 2; k <= N; k <<= 1) {
		// internal loop for comparator internal stages
		for (j = k >> 1; j > 0; j = j >> 1)
			bitonic_sort_step<<<blocks, threads>>>(d_a, j, k);
	}
	cudaEventRecord(stop);
	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);
	printf("GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

	// recover data
	cudaMemcpy(a, d_a, nBytes, cudaMemcpyDeviceToHost);

	// print & check
	if (N < 100) {
		printf("GPU:\n");
		for (int i = 0; i < N; ++i)
			printf("%d  ", a[i]);
		printf("\nCPU:\n");
		for (int i = 0; i < N; ++i)
			printf("%d  ", b[i]);
	}
	else {
		for (int i = 0; i < N; ++i) {
			if (a[i] != b[i]) {
				printf("ERROR a[%d] != b[%d]  (a[i] = %d  -  b[i] = %d\n", i,i, a[i],b[i]);
				break;
			}
		}
	}

	cudaFree(d_a);
	exit(0);
}

In [ ]:
# for debugging (small vectors)
#!nvcc -arch=sm_37 -DDEBUG sorting/bitonic.cu -o bitonic

# for test on big vectors
!nvcc -arch=sm_37 sorting/bitonic.cu -o bitonic
!./bitonic

# ✅ QuickSort

In [ ]:
%%writefile sorting/quicksort.cu

#include <iostream>
#include <cstdio>
#include "../../utils/common.h"

#define MAX_DEPTH       16
#define INSERTION_SORT  32

////////////////////////////////////////////////////////////////////////////////
// Selection sort used when depth gets too big or the number of elements drops
// below a threshold.
////////////////////////////////////////////////////////////////////////////////
__device__ void selection_sort(unsigned int *data, int left, int right) {
    for (int i = left ; i <= right ; ++i) {
        unsigned min_val = data[i];
        int min_idx = i;

        // Find the smallest value in the range [left, right].
        for (int j = i+1 ; j <= right ; ++j) {
            unsigned val_j = data[j];

            if (val_j < min_val) {
                min_idx = j;
                min_val = val_j;
            }
        }

        // Swap the values.
        if (i != min_idx) {
            data[min_idx] = data[i];
            data[i] = min_val;
        }
    }
}

////////////////////////////////////////////////////////////////////////////////
// Very basic quicksort algorithm, recursively launching the next level.
////////////////////////////////////////////////////////////////////////////////
__global__ void cdp_simple_quicksort(unsigned int *data, int left, int right, int depth) {
    // If we're too deep or there are few elements left, we use an insertion sort...
    if (depth >= MAX_DEPTH || right-left <= INSERTION_SORT) {
        selection_sort(data, left, right);
        return;
    }

    unsigned int *lptr = data+left;
    unsigned int *rptr = data+right;
    unsigned int  pivot = data[(left+right)/2];

    // Do the partitioning.
    while (lptr <= rptr) {
        // Find the next left- and right-hand values to swap
        unsigned int lval = *lptr;
        unsigned int rval = *rptr;

        // Move the left pointer as long as the pointed element is smaller than the pivot.
        while (lval < pivot) {
            lptr++;
            lval = *lptr;
        }

        // Move the right pointer as long as the pointed element is larger than the pivot.
        while (rval > pivot) {
            rptr--;
            rval = *rptr;
        }

        // If the swap points are valid, do the swap!
        if (lptr <= rptr) {
            *lptr++ = rval;
            *rptr-- = lval;
        }
    }

    // Now the recursive part
    int nright = rptr - data;
    int nleft  = lptr - data;

    // Launch a new block to sort the left part.
    if (left < (rptr-data)) {
        cudaStream_t s;
        cudaStreamCreateWithFlags(&s, cudaStreamNonBlocking);
        cdp_simple_quicksort<<< 1, 1, 0, s >>>(data, left, nright, depth+1);
        cudaStreamDestroy(s);
    }

    // Launch a new block to sort the right part.
    if ((lptr-data) < right) {
        cudaStream_t s1;
        cudaStreamCreateWithFlags(&s1, cudaStreamNonBlocking);
        cdp_simple_quicksort<<< 1, 1, 0, s1 >>>(data, nleft, right, depth+1);
        cudaStreamDestroy(s1);
    }
}

////////////////////////////////////////////////////////////////////////////////
// Call the quicksort kernel from the host.
////////////////////////////////////////////////////////////////////////////////
void run_qsort(unsigned int *data, unsigned int nitems) {
    // Prepare CDP for the max depth 'MAX_DEPTH'.
    CHECK(cudaDeviceSetLimit(cudaLimitDevRuntimeSyncDepth, MAX_DEPTH));

    // Launch on device
    int left = 0;
    int right = nitems-1;
    std::cout << "Launching kernel on the GPU" << std::endl;
    cdp_simple_quicksort<<< 1, 1 >>>(data, left, right, 0);
    CHECK(cudaDeviceSynchronize());
}

////////////////////////////////////////////////////////////////////////////////
// Initialize data on the host.
////////////////////////////////////////////////////////////////////////////////
void initialize_data(unsigned int *dst, unsigned int nitems) {
    // Fixed seed for illustration
    srand(2047);

    // Fill dst with random values
    for (unsigned i = 0 ; i < nitems ; i++)
        dst[i] = rand() % nitems ;
}

////////////////////////////////////////////////////////////////////////////////
// Verify the results.
////////////////////////////////////////////////////////////////////////////////
void check_results(int n, unsigned int *results_d) {
    unsigned int *results_h = new unsigned[n];
    CHECK(cudaMemcpy(results_h, results_d, n*sizeof(unsigned), cudaMemcpyDeviceToHost));

    for (int i = 1 ; i < n ; ++i)
        if (results_h[i-1] > results_h[i]) {
            std::cout << "Invalid item[" << i-1 << "]: " << results_h[i-1] << " greater than " << results_h[i] << std::endl;
            exit(EXIT_FAILURE);
        }

    std::cout << "OK" << std::endl;
    delete[] results_h;
}

////////////////////////////////////////////////////////////////////////////////
// Main entry point.
////////////////////////////////////////////////////////////////////////////////
int main(int argc, char **argv) {
    int num_items = 1024*1024;
    bool verbose = false;

    // Get device properties
    int device = 0;
    
    cudaSetDevice(device);

    // Create input data
    unsigned int *h_data = 0;
    unsigned int *d_data = 0;

    // Allocate CPU memory and initialize data.
    std::cout << "Initializing data:" << std::endl;
    h_data =(unsigned int *)malloc(num_items*sizeof(unsigned int));
    initialize_data(h_data, num_items);

    if (verbose) {
        for (int i=0 ; i<num_items ; i++)
            std::cout << "Data [" << i << "]: " << h_data[i] << std::endl;
    }

    // Allocate GPU memory.
    CHECK(cudaMalloc((void **)&d_data, num_items * sizeof(unsigned int)));
    CHECK(cudaMemcpy(d_data, h_data, num_items * sizeof(unsigned int), cudaMemcpyHostToDevice));

    // Execute
    std::cout << "Running quicksort on " << num_items << " elements" << std::endl;
    run_qsort(d_data, num_items);

    // Check result
    std::cout << "Validating results: ";
    check_results(num_items, d_data);

    free(h_data);
    CHECK(cudaFree(d_data));

    // cudaDeviceReset causes the driver to clean up all state. While
    // not mandatory in normal operation, it is good practice.  It is also
    // needed to ensure correct operation when the application is being
    // profiled. Calling cudaDeviceReset causes all profile data to be
    // flushed before the application exits
    cudaDeviceReset();
    exit(EXIT_SUCCESS);
}


In [ ]:
# for test on big vectors

!nvcc -arch=sm_37 -dc sorting/quicksort.cu 
!nvcc -arch=sm_37 quicksort.o -o quicksort
!./quicksort